pip install tensorflow keras matplotlib nltk


https://www.kaggle.com/datasets/adityajn105/flickr8k
dataset Flicker

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import numpy as np
import os

model = InceptionV3(weights='imagenet')
cnn_model = Model(inputs=model.input, outputs=model.layers[-2].output) 

In [ ]:
def extract_features(directory):
    features = {}
    for img_name in os.listdir(directory):
        img_path = os.path.join(directory, img_name)
        img = image.load_img(img_path, target_size=(299, 299))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        feature = cnn_model.predict(img)
        features[img_name] = feature
    return features


In [ ]:
import string
import re

def clean_caption(caption):
    caption = caption.lower()
    caption = re.sub(r"[^a-zA-Z ]", "", caption)
    caption = 'startseq ' + caption + ' endseq'
    return caption

# Clean and tokenize all captions